In [48]:
import pandas as pd
import numpy as np
import seaborn as sns

## Testes iniciais

In [49]:
#vamos consultar duas bases diferentes para entender seu formato

df = pd.read_csv('https://www.senado.gov.br/transparencia/LAI/verba/despesa_ceaps_2022.csv',
                 skiprows=1, #A primeira é irrelevante
                 delimiter=';', #Precisamos passar o delimitador, por padrão pandas usa virgula
                 encoding='latin') #Setar o enconding por conta dos acentos
                 
df_2 = pd.read_csv('https://www.senado.gov.br/transparencia/LAI/verba/despesa_ceaps_2021.csv',
                 skiprows=1, #A primeira é irrelevante
                 delimiter=';', #Precisamos passar o delimitador, por padrão pandas usa virgula
                 encoding='latin')#Setar o enconding por conta dos acentos

In [50]:
#As bases possuem os mesmos campos, então vamos tentar empilhar todos os anos
print(df.columns.values)

['ANO' 'MES' 'SENADOR' 'TIPO_DESPESA' 'CNPJ_CPF' 'FORNECEDOR' 'DOCUMENTO'
 'DATA' 'DETALHAMENTO' 'VALOR_REEMBOLSADO' 'COD_DOCUMENTO']


In [51]:
#Primeiro, vamos criar um df vazio com as colunas:
col_names = df.columns

df_conc = pd.DataFrame(columns=col_names)

df_conc

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO


In [52]:
#Criar o loop para consolidar as bases
#Estão disponiveis dados desde 2008 até 2022, então vamos criar uma lista com esses valores

#Gerar lista númerica para ter um log simples do union das bases

anos = []
anos = [ i for i in range(2008,2023) ]

get = 'https://www.senado.gov.br/transparencia/LAI/verba/despesa_ceaps_'



In [53]:
for ano in anos:
    
    ano_str = get + str(ano) + '.csv'
    
    df_loop = pd.read_csv(ano_str,
                     skiprows=1, #Não trazer o header
                     delimiter=';', #Precisamos passar o delimitador, por padrão pandas usa virgula
                     encoding='latin')#Setar o enconding por conta dos acentos
    
    
    print('Ano de consulta : ' + str(ano))
    print(df_loop.columns.values)
    print('-------------------------------------------')
    print('colunas do df')
    print(df_conc.columns.values)
    
    
    df_conc = pd.concat([df_conc,df_loop])

Ano de consulta : 2008
['ANO' 'MES' 'SENADOR' 'TIPO_DESPESA' 'CNPJ_CPF' 'FORNECEDOR' 'DOCUMENTO'
 'DATA' 'DETALHAMENTO' 'VALOR_REEMBOLSADO' 'COD_DOCUMENTO']
-------------------------------------------
colunas do df
['ANO' 'MES' 'SENADOR' 'TIPO_DESPESA' 'CNPJ_CPF' 'FORNECEDOR' 'DOCUMENTO'
 'DATA' 'DETALHAMENTO' 'VALOR_REEMBOLSADO' 'COD_DOCUMENTO']
Ano de consulta : 2009
['ANO' 'MES' 'SENADOR' 'TIPO_DESPESA' 'CNPJ_CPF' 'FORNECEDOR' 'DOCUMENTO'
 'DATA' 'DETALHAMENTO' 'VALOR_REEMBOLSADO' 'COD_DOCUMENTO']
-------------------------------------------
colunas do df
['ANO' 'MES' 'SENADOR' 'TIPO_DESPESA' 'CNPJ_CPF' 'FORNECEDOR' 'DOCUMENTO'
 'DATA' 'DETALHAMENTO' 'VALOR_REEMBOLSADO' 'COD_DOCUMENTO']
Ano de consulta : 2010
['ANO' 'MES' 'SENADOR' 'TIPO_DESPESA' 'CNPJ_CPF' 'FORNECEDOR' 'DOCUMENTO'
 'DATA' 'DETALHAMENTO' 'VALOR_REEMBOLSADO' 'COD_DOCUMENTO']
-------------------------------------------
colunas do df
['ANO' 'MES' 'SENADOR' 'TIPO_DESPESA' 'CNPJ_CPF' 'FORNECEDOR' 'DOCUMENTO'
 'DATA' 'DETA

In [54]:
df_conc.reset_index(drop=True,inplace=True)

## Data Wrangling 

In [55]:
df_conc_bkp = df_conc
df_conc.isna().sum()

ANO                       0
MES                       0
SENADOR                   0
TIPO_DESPESA              0
CNPJ_CPF               3144
FORNECEDOR             3144
DOCUMENTO             24085
DATA                   3144
DETALHAMENTO         131642
VALOR_REEMBOLSADO         0
COD_DOCUMENTO             2
dtype: int64

In [56]:
#Substituir alguns valores de data pela data que está na coluna de documento
df_conc.loc[df_conc['DATA'].str.len() > 10,'DATA'] = df_conc.loc[df_conc['DATA'].str.len() > 10,'DOCUMENTO'].str.split(';',expand=True)[1]
df_conc.loc[df_conc['DATA'].str.len() > 10,'DATA'] = df_conc.loc[df_conc['DATA'].str.len() > 10,'DATA'].str.split('"',expand=True)[0]

#Tratar a coluna valor reembolsado para transforma-lo em float
df_conc['VALOR_REEMBOLSADO'] = df_conc['VALOR_REEMBOLSADO'].str.replace('1\r\n','')
df_conc['VALOR_REEMBOLSADO'] = df_conc['VALOR_REEMBOLSADO'].str.replace(',','.')
df_conc['VALOR_REEMBOLSADO'] = df_conc['VALOR_REEMBOLSADO'].astype('float')

#Tratamento dos textos e data
df_conc.loc[df_conc['DETALHAMENTO'].isnull() == True,'DETALHAMENTO'] = 'SEM DETALHAMENTO'
df_conc['MES'] = df_conc['MES'].astype(str).str.zfill(2)
df_conc['DT_DATA'] = df_conc['DIA'] + df_conc['MES'].astype(str) + df_conc['ANO'].astype(str)

In [62]:
int_dict = {'ANO':'int16','MES':'int16'}
cat_dict = {'SENADOR':'category','TIPO_DESPESA':'category','FORNECEDOR':'category'}
str_dict = {'DETALHAMENTO':'string'}
drop_col = ['COD_DOCUMENTO','DOCUMENTO','CNPJ_CPF']


In [83]:
df_conc = df_conc.astype(int_dict)
df_conc = df_conc.astype(cat_dict)
df_conc = df_conc.astype(str_dict)
df_conc = df_conc.drop(drop_col,axis=1)
df_conc['DT_DATA'] = pd.to_datetime(df_conc['DT_DATA'], format='%d%m%Y',errors='coerce')


In [85]:
#deletando os nulos restantes perde-se apenas 1% da base.
df_conc.isna().sum() / df_conc.count() * 100

ANO                  0.000000
MES                  0.000000
SENADOR              0.000000
TIPO_DESPESA         0.000000
FORNECEDOR           1.040464
DATA                 1.040464
DETALHAMENTO         0.000000
VALOR_REEMBOLSADO    0.000000
DIA                  1.040464
DT_DATA              1.284144
dtype: float64

In [86]:
df_final = df_conc.dropna()

In [87]:
#Exportar Base
df_final.to_excel('base_consolidada.xlsx')